In [3]:
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt
from statistics import mean, median, mode, stdev
import ast
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [4]:
Movie = pd.read_csv('movie_collection_data_revenuedesc.csv',na_filter=True, na_values=['[]','NaN'])
Movie.head()

FileNotFoundError: [Errno 2] File movie_collection_data_revenuedesc.csv does not exist: 'movie_collection_data_revenuedesc.csv'

# Dropping invalid Data

In [ ]:
## Drop INVALID DATA
Numericlist = ['Budget','Popularity','VoteAvg','VoteCount','Revenue','Runtime']
for item in Numericlist:
    Movie[str(item)] = pd.to_numeric(Movie[str(item)], errors='coerce').fillna(0)
Movie_CleanedData = Movie.dropna(subset=['ReleaseDate','SpokenLanguage','ProductionCompany','ProductionCountry','GenreList'])
Movie_CleanedData = Movie_CleanedData[~(Movie_CleanedData['Budget']<281)]
Movie_CleanedData = Movie_CleanedData[~(Movie_CleanedData['Revenue']==0)]
Movie_CleanedData.drop_duplicates(subset="Movie_name", inplace =True)
Movie_CleanedData.index = pd.RangeIndex(len(Movie_CleanedData.index))
Movie_CleanedData.index = range(len(Movie_CleanedData.index))



In [ ]:
Movie_CleanedData.info()

## Calculate and Insert Profit Variable

In [ ]:
Profit=[]
for i in range(len(Movie_CleanedData)):
    Profit.append(Movie_CleanedData.loc[i,'Revenue']-Movie_CleanedData.loc[i,'Budget'])
Movie_CleanedData.insert(10,"Profit",Profit,True)

## Create new column for release year and month

In [ ]:
Movie_CleanedData['Year'] = pd.DatetimeIndex(Movie_CleanedData['ReleaseDate']).year
Movie_CleanedData['Month'] = pd.DatetimeIndex(Movie_CleanedData['ReleaseDate']).month

In [ ]:
#change string in list format to list
for i in range(len(Movie_CleanedData)):
    x = ast.literal_eval(Movie_CleanedData.loc[i,'GenreList'])
    Movie_CleanedData.at[i,'GenreList'] = x
    
    x = ast.literal_eval(Movie_CleanedData.loc[i,'ProductionCompany'])
    Movie_CleanedData.at[i,'ProductionCompany'] = x
    
    x = ast.literal_eval(Movie_CleanedData.loc[i,'ProductionCountry'])
    Movie_CleanedData.at[i,'ProductionCountry'] = x
    
    x = ast.literal_eval(Movie_CleanedData.loc[i,'SpokenLanguage'])
    Movie_CleanedData.at[i,'SpokenLanguage'] = x

In [ ]:
Movie_CleanedData.head(1000)

In [ ]:
Movie_CleanedData.info()

## Create Columns for Categorical Data


In [ ]:
# #change string in list format to list
# for i in range(len(Movie_CleanedData)):
#     x = ast.literal_eval(Movie_CleanedData.loc[i,'GenreList'])
#     Movie_CleanedData.at[i,'GenreList'] = x
    
#     x = ast.literal_eval(Movie_CleanedData.loc[i,'ProductionCompany'])
#     Movie_CleanedData.at[i,'ProductionCompany'] = x
    
#     x = ast.literal_eval(Movie_CleanedData.loc[i,'ProductionCountry'])
#     Movie_CleanedData.at[i,'ProductionCountry'] = x
    
#     x = ast.literal_eval(Movie_CleanedData.loc[i,'SpokenLanguage'])
#     Movie_CleanedData.at[i,'SpokenLanguage'] = x

    
# #count max number each category, to create number of columns
# genrecount,production_company_count,production_country_count,spoken_language_count = 0,0,0,0

# for i in range(len(Movie_CleanedData)):
#     if len(Movie_CleanedData.loc[i,'GenreList'])>genrecount:
#         genrecount = len(Movie_CleanedData.loc[i,'GenreList'])
#     if len(Movie_CleanedData.loc[i,'ProductionCompany'])>production_company_count:
#         production_company_count = len(Movie_CleanedData.loc[i,'ProductionCompany'])
#     if len(Movie_CleanedData.loc[i,'ProductionCountry'])>production_country_count:
#         production_country_count = len(Movie_CleanedData.loc[i,'ProductionCountry'])
#     if len(Movie_CleanedData.loc[i,'SpokenLanguage'])>spoken_language_count:
#         spoken_language_count = len(Movie_CleanedData.loc[i,'SpokenLanguage'])
        
       
# print(genrecount)
# print(production_company_count)
# print(production_country_count)
# print(spoken_language_count)
        
# #to catch and ignore list out of index, replace with NONE.
# def catch(var,i):
#     try:
#         return var[i]
#     except Exception as e:
#         return None
# def catch2(var,i):
#     try:
#         return var[i][0]
#     except Exception as e:
#         return None
    
# #replace
# for i in range(1,genrecount+1):
#     Movie_CleanedData.loc[:, 'Genre'+str(i)] = [catch(x, i-1) for x in Movie_CleanedData.GenreList]
    
# for i in range(1,production_company_count+1):
#     Movie_CleanedData.loc[:, 'ProductionCompany'+str(i)] = [catch2(x, i-1) for x in Movie_CleanedData.ProductionCompany]
    
# for i in range(1,production_country_count+1):    
#     Movie_CleanedData.loc[:, 'ProductionCountry'+str(i)] = [catch(x, i-1) for x in Movie_CleanedData.ProductionCountry]

# for i in range(1,spoken_language_count+1):    
#     Movie_CleanedData.loc[:, 'SpokenLanguage'+str(i)] = [catch(x, i-1) for x in Movie_CleanedData.SpokenLanguage]

In [ ]:
Movie_CleanedData.describe()

## EXPLORATORY ANALYSIS

## Numeric Data

In [ ]:
Q1 = np.quantile(Movie_CleanedData.VoteCount, 0.25)
for i in range(len(Movie_CleanedData)):
    if Movie_CleanedData.loc[i,'VoteCount']<Q1:
        Movie_CleanedData.loc[i,'VoteCount'] = np.NaN
        Movie_CleanedData.loc[i,'VoteAvg'] = np.NaN

In [ ]:
#Plotting VoteCount versus Profit
fig = plt.gcf()
fig.set_size_inches(10, 6)
Movie_CleanedData.VoteCount.dropna()
plt.title("Profit vs VoteCount", size=24)
sb.scatterplot(data=Movie_CleanedData, x="VoteCount", y="Profit")

In [ ]:
#Plotting VoteAvg versus Profit
fig = plt.gcf()
fig.set_size_inches(10, 6)
Movie_CleanedData.VoteAvg.dropna()
plt.title("Profit vs VoteAvg", size=24)
sb.scatterplot(data=Movie_CleanedData, x="VoteAvg", y="Profit")

In [ ]:
NumericData =pd.DataFrame(Movie_CleanedData[['Budget','Popularity','VoteAvg','VoteCount','Revenue','Profit','Runtime','Year','Month']])
NumericData.head()

In [ ]:
f, axes = plt.subplots(9, 3, figsize=(36, 40))

count = 0
for var in NumericData:
    sb.boxplot(data = NumericData[var], orient = "h", ax = axes[count,0])
    sb.histplot(data = NumericData[var], ax = axes[count,1])
    sb.violinplot(data = NumericData[var], orient = "h", ax = axes[count,2])
    count += 1

In [ ]:

# Correlation Matrix
print(NumericData.corr())

# Heatmap of the Correlation Matrix
f = plt.figure(figsize=(12, 12))
sb.heatmap(NumericData.corr(), vmin = -1, vmax = 1, linewidths = 1,
           annot = True, fmt = ".2f", annot_kws = {"size": 18}, cmap = "RdBu")

In [ ]:
sb.pairplot(data = NumericData)

## Categorical Data

In [ ]:
Genre = Movie_CleanedData.set_index('Movie_name').GenreList.apply(pd.Series).stack()
Genre = pd.get_dummies(Genre).groupby(level=0).sum()
Genre = Genre.replace(0,np.nan)
Movie_CleanedData = Movie_CleanedData.set_index('Movie_name')
Genre['Profit'] = Movie_CleanedData.Profit
Genre['Year'] = Movie_CleanedData.Year

In [ ]:
Genre.head()

In [ ]:
f, axes = plt.subplots(len(Genre.columns)-1, 1, figsize=(40, 150))
count = 0
for var in Genre.columns:
    if var == 'Profit':
        break
    sb.boxplot(y = Genre[var], x = Genre['Profit'], orient = "h", ax = axes[count])
    count+=1

In [ ]:
df = Genre.melt(id_vars=['Year',"Profit"], var_name="Genre",value_name="Value")
df = df.sample(frac=0.25,axis='rows')
df.info()

In [ ]:
df.head(100)

In [ ]:
sb.relplot(x=df['Year'],y=df['Profit'],hue=df['Genre'], size = pd.qcut(df["Profit"], 4, precision=1),
                sizes=[0.01, 1, 25, 100], palette="bright", height = 10)


## For Production Companies, there are over 7000 unique companies. To deal with this overload of categorical data, we bin the companies into 4 categories based on their movie count frequency. 

In [ ]:
ProComDic ={}
for i in range(len(Movie_CleanedData)):
    for j in range(len(Movie_CleanedData.ProductionCompany[i])): #every element in genrelist
        if Movie_CleanedData.ProductionCompany[i][j] not in ProComDic: #if genre is not in dictionary, initialize genre: 1
            ProComDic[str(Movie_CleanedData.ProductionCompany[i][j])] = 1
        elif Movie_CleanedData.ProductionCompany[i][j] in ProComDic:
            ProComDic[str(Movie_CleanedData.ProductionCompany[i][j])] +=1

In [ ]:
print(ProComDic)

In [5]:
ProComBin={}


others = [x[0] for x in ProComDic.items() if x[1]<=20]
small = [x[0] for x in ProComDic.items() if 20<x[1]<=75]
medium = [x[0] for x in ProComDic.items() if 75<x[1]<=150]
large = [x[0] for x in ProComDic.items() if 150<x[1]]
        

NameError: name 'ProComDic' is not defined

In [ ]:
Movie_CleanedData.head()

In [ ]:
ProCom = Move_CleanedData
ProCom['ProductionCompany'] = ProCom.where()

# ProCom = pd.get_dummies(Genre).groupby(level=0).sum()
# ProCom = ProCom.replace(0,np.nan)
# Movie_CleanedData = Movie_CleanedData.set_index('Movie_name')
# ProCom['Profit'] = Movie_CleanedData.Profit
# ProCom['Year'] = Movie_CleanedData.Year